<div style="direction:rtl;text-align:right">
    
## تحليل النصوص

<div style="direction:rtl;text-align:right">
في هذا الدرس، سوف نتعرف على مجال تحليل اللغة الطبيعية واستخدام ما تعلمناه من تقنيات بناء النماذج والتحقق منها لأداء العديد من المهام كتصنيف النصوص إلى فئات معينة أو تحليل المشاعر وما إلى ذلك.

<div style="direction:rtl;text-align:right">
الكثير من تقنيات تحليل النصوص تقوم على استخدام نفس المبادئ التي تعرفنا عليها في الدروس السابقة، لكن التحدي هو كيف نقوم بتحويل النصوص إلى مصفوفات من الأرقام لتتمكن خوارزميات تحليل البيانات من التعامل معها؟ في هذا الدرس سوف نستخدم أحد أهم طرق تمثيل البيانات وهي: Bag of words أو سلة الكلمات. في هذا التمثيل، نقوم بتمثيل كل نص (أو مستند، أو تغريدة، ..) بمجموعة من الأرقام التي تمثل تكرار الكلمات داخل هذا النص. (صورة توضيحية)

<div style="direction:rtl;text-align:right">
لنبدأ باستيراد المكتبات اللازمة ونرى كيف يمكن تنفيذ هذا الأمر عمليا:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from sklearn import datasets

<div style="direction:rtl;text-align:right">
    
### معالجة النصوص

<div style="direction:rtl;text-align:right">
هناك مجموعة من الخطوات الأساسية لمعالجة النصوص قبل تنفيذ أي عمليات عليها وسنقوم باستعراضها وشرحها بشكل موجز قبل تنفيذها عملياً. خطوات معالجة النصوص يمكن تلخيصها كالتالي: <br><br>
    - حذف علامات الترقيم والأرقام والرموز الخاصة: قد تحتاج لعمل هذه الخطوة في أغلب تطبيقات معالجة اللغة الطبيعية. فكل علامات الترقيم (مثل الفواصل والنقاط وعلامات الاستفهام والتعجب الخ) أو الأرقام (مثلا 4 أو ٤) أو الرموز الخاصة (مثل # أو @ أو $) لا تقدم -عموما- ميزة في تمييز أنواع النصوص. الاستثناء الوحيد هو حين تكون الأرقام أو علامات الترقيم مهمة لنا أثناء التحليل لاحقاً، مثل لو كنا نعمل على تحليل استخدام علامات الترقيم بين عدد من الكتاب أو عدد من الروايات. <br>
    - توحيد النص (Text Normalization) وفي هذه الخطوة سنقوم بتوحيد أساليب التعبير في النص. مثلاً، بدلاً من استخدام خمس رموز للهمزة (أ، ا، إ، ئ، ء)، سوف نستدلها جميعاً ب (ا). فقط. الهدف من هذه الخطوة هو توحيد النص ليسهل دمج الكلمات المتشابهة. <br>
    - حذف الكلمات المستبعدة (stop words): في هذه الخطوة سنقوم بحذف الكلمات العامة والتي لا تعطي أي ميزة أو معنى حين نستخدم سلة الكلمات. فمثلاً: في، عن، ذلك، لكن، وغيرها من الكلمات لا تقدم أي معنى محدد بل هي عامة جداً. في الانجليزية، نجد أن كلمات مثل the, in, but, for, وغيرها تقوم بنفس الدور. <br>
    - استخدام جذور الكلمات (Stemming). هذه الخطوة لا تعتبر ضرورية بل تستخدم أحياناً بحسب الحاجة وبحسب التطبيق. في هذه الخطوة، سنقوم باستخدام جذور الكلمات بدلاً عن الكلمات. فمثلاً بدلاً من : الحروب، حارب، حرب، حربي وغيرها، سنقوم باستخدام جذرها وهو "حرب". هذه الخطوة مفيدة جداً في توحيد النص (كما ذكرنا أعلاه) لكن قد لا تكون مناسبة لكثير من البيانات حين يكون شكل الكلمة عامل مهم جداً. <br>
        - فصل الكلمات Word Tokenization: في هذه الخطوة، نقوم بفصل الكلمات عن بعضها البعض لنتعامل مع كل كلمة على حدة. فبدلاً من "لندن عاصمة بريطانيا" سيكون لدينا القائمة التالية: ["لندن", "عاصمة", "بريطانيا"]. قد تبدو هذه الخطوة منطقية وسهلة جداً حين نستخدم المسافة فحسب لفصل الكلمات عن بعضها البعض. لكن الواقع أن المسافات قد لاتكون كافية للعمل مع جميع اللغات التي يختلف نظام الحروف فيها أو مع التطبيقات التي تعمد إلى تجاهل المسافات بين الكلمات ناهيك عن الأخطاء الإملائية التي قد تتجاهل المسافات.<br>

<div style="direction:rtl;text-align:right">
هناك المزيد من الخطوات والتقنيات لمعالجة النصوص ولكن سنكتفي بهذا القدر لنرى كيف يمكن تنفيذ بعض هذه الخطوات في تحليل النصوص.

<div style="direction:rtl;text-align:right">
سنقوم في هذا الدرس بتقسيم المقالات أو الكلمات الجديدة بحسب نوعها هل هي سياسية أو اقتصادية أو تقنية. قبل ذلك، نحتاج إلى بيانات لنتعلم منها ما الذي يفصل هذه المجالات وما الكلمات التي تميز مجالاً عن غيره.
    
    نحصل على ملف البيانات المطلوب من خلال هذا الرابط: 
https://drive.google.com/file/d/1yyKzYwl7O_eSj7hyOirR7i8F6nGuv-J5/view?usp=sharing

In [3]:
df = pd.read_csv('arabic_dataset_classifiction.csv')

In [4]:
df.columns = ['text', 'category']

In [5]:
df.head()

,text,category
0,بين أستوديوهات ورزازات وصحراء مرزوكة وآثار ولي...,0
1,قررت النجمة الأمريكية أوبرا وينفري ألا يقتصر ع...,0
2,أخبارنا المغربية الوزاني تصوير الشملالي ألهب ا...,0
3,اخبارنا المغربية قال ابراهيم الراشدي محامي سعد...,0
4,تزال صناعة الجلود في المغرب تتبع الطريقة التقل...,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111728 entries, 0 to 111727
Data columns (total 2 columns):
text        108789 non-null object
category    111728 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.7+ MB


In [7]:
df.shape

(111728, 2)

<div style="direction:rtl;text-align:right">
تحتوي البيانات أعلاه على 111,728 مقالة ولكل مقالة لدينا عمود يحتوي على نص المقالة وآخر يحتوي على رقم يبين لنا نوع المقالة. لاحظ أن لدينا بعض البيانات المفقودة من خلال أمر info أعلاه. لكن لمعرفة العدد الدقيق للبيانات المفقودة سنقوم بكتابة الأمر التالي: 

In [8]:
df.isnull().sum()

text        2939
category       0
dtype: int64

<div style="direction:rtl;text-align:right">
يخبرنا الأمر أعلاه بأن حوالي 3000 مقالة ينقصها النص. للتعامل مع هذه المقالات المفقودة، سنقوم بحذفها فحسب.

In [9]:
df.dropna(inplace=True)

<div style="direction:rtl;text-align:right">
صحيح أن لدينا عمود يسمى target يبين نوع كل مقالة ولكن القيم فيه عبارة عن أرقام تمثل الأنواع ولايوجد لدينا أي فكرة عن أي منها يمثل أي نوع. لمعرفة ذلك، سنقوم بطباعة ثلاث مقالات عشوائية من كل نوع لنأخذ فكرة.

In [10]:
df['category'].value_counts()

4    43675
3    20485
1    16728
2    14165
0    13736
Name: category, dtype: int64

<div style="direction:rtl;text-align:right">
في المثال التالي، سنستعرض عملية فصل الكلمات باستخدام أمر word_tokenize.

In [11]:
import nltk

In [12]:
from nltk.tokenize import word_tokenize
raw = df['text'][0]
tokenized = word_tokenize(raw)

In [13]:
print('======= Raw Text =======\n')
print(raw)
print('\n\n======= Tokenized ======\n')
print(tokenized)

======= Raw Text =======

بين أستوديوهات ورزازات وصحراء مرزوكة وآثار وليلي ثم الرباط والبيضاء انتهى المخرج المغربي سهيل بن بركة من تصوير مشاهد عمله السينمائي الجديد الذي خصصه لتسليط الضوء عن حياة الجاسوس الإسباني دومينغو باديا الذي عاش فترة من القرن التاسع عشر بالمغرب باسم علي باي هذا الفيلم الذي اختار له مخرجه عنوان حلم خليفة يصور حياة علي باي العباسي الذي ما زال أحد أحياء طنجة يحمل اسمه عاش حياة فريدة متنكرا بشخصية تاجر عربي من سلالة الرسول صلى الله عليه وسلم فيما كان يعمل جاسوسا لحساب إسبانيا وكشف مخرج الفيلم سهيل بن بركة في تصريح لهسبريس أن الفيلم السينمائي دخل مرحلة التوضيب التي تتم خارج المغرب مبرزا أن الفيلم الذي يروي حياة الجاسوس الإسباني دومينغو باديا منذ أن قرر من طنجة بدء رحلاته نحو عدد من المناطق في العالم الإسلامي بداية القرن العشرين سيكون جاهزا بعد شهرين ويجمع الفيلم السينمائي عددا من الممثلين من مختلف الجنسيات واختار لدور البطولة الممثلة السينمائية الإيطالية كارولينا كريشنتيني للقيام بدور الإنجليزية الليدي هستر ستانهوب التي اشتهرت في الكتب الغربية بـ زنوبيا والتي عاشت بد

<div style="direction:rtl;text-align:right">
لاحظ أن البيانات لا توجد فيها أي علامات ترقيم أو رموز خاصة بالتالي قد لا نكون بحاجة لتطبيق جميع الخطوات في تحليل هذه البيانات النظيفة. بالتالي يمكننا أن نرى كيف يمكن تحويل هذه البيانات إلى مصفوفة سلة الكلمات.

<div style="direction:rtl;text-align:right">
لإنشاء سلة الكلمات، سنحتاج لاستخدام الأمر CountVectorizer من مكتبة سايكت ليرن. هذا الأمر يبسط علينا المهمة إلى حد كبير. كل ما نحتاجه لاستخدام هذا الأمر هو تحديد المتغيرات مثل طريقة فصل الكلمات والحد الأدنى أو الأعلى لتكرار الكلمة وما إلى ذلك ومن ثم سيقوم الأمر بتحويل النصوص إلى مصفوفة تمثل تكرار كل كلمة داخل كل مقالة.

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
vectorizer = CountVectorizer(encoding='utf-8', 
                             tokenizer=word_tokenize,  # we can leave this to use the default tokenizer
                             min_df=.01,                # minimum freq of words to be included: words has to appear in 10% of articles
                             max_df=.95)                # maximum freq of words to be included: words in more than 90% of articles will NOT be included (too frequent)

<div style="direction:rtl;text-align:right">
لإنشاء سلة الكلمات سنقوم باستخدام fit_transform كما نفعل دائما (fit->transform)

In [16]:
vectorizer.fit(df.text)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=0.95, max_features=None, min_df=0.01,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function word_tokenize at 0x121606048>,
                vocabulary=None)

In [17]:
text_doc_matrix = vectorizer.transform(df.text)

In [18]:
text_doc_matrix.shape

(108789, 2975)

<div style="direction:rtl;text-align:right">
عدد صفوف المصفوفة يجب أن يكون هو نفس عدد المقالات. أما عدد الأعمدة فلدينا 156 عمود أي أن لدينا 156 كلمة مختلفة. لطباعة الكلمات بشكل يسهل علينا قراءتها سنقوم أولاً باستدعاء الكلمات ثم بطباعتها في سطر واحد مع استخدام الخط العمودي الفاصل لفصل كل كلمة عن الأخرى: 

In [19]:
' | '.join(vectorizer.get_feature_names())

'» | ، | آخر | آخرون | آخرين | آسفي | آلاف | آليات | آيت | أبدى | أبرز | أبرزها | أبريل | أبطال | أبناء | أبو | أتمنى | أثار | أثارت | أثر | أثناء | أجرى | أجل | أجواء | أحد | أحداث | أحدهما | أحزاب | أحسن | أحمد | أخبار | أخبارنا | أخذ | أخرى | أخنوش | أخيرا | أداء | أدلى | أدى | أربع | أربعة | أرض | أرضية | أزمة | أزيد | أسابيع | أساس | أساسا | أساسيا | أساسية | أسباب | أسبوع | أستاذ | أسرة | أسعار | أسفرت | أسماء | أشار | أشارت | أشخاص | أشرف | أشغال | أشهر | أصبح | أصبحت | أصحاب | أصدرت | أصل | أصيب | أضاف | أطلس | أطلق | أعتقد | أعضاء | أعطى | أعلن | أعلنت | أعلى | أعمال | أعماله | أغلب | أغنية | أفاد | أفادت | أفراد | أفضل | أفق | أقدم | أقرب | أقل | أقوى | أكادير | أكبر | أكتوبر | أكثر | أكد | أكدت | ألا | ألعاب | ألف | ألفا | ألمانيا | أم | أما | أمام | أمر | أمريكا | أمس | أمل | أمم | أمن | أمني | أمنية | أموال | أمين | أمينة | أن | أنا | أنحاء | أندية | أنشطة | أنظار | أنفسهم | أننا | أنني | أنه | أنها | أنهم | أنهما | أنهى | أهداف | أهم | أهمية | أو | أوربا | أورو | أوروبا |

<div style="direction:rtl;text-align:right">
بعد أن أصبح لدينا مصفوفة تمثل البيانات، يمكننا الآن استخدام النماذج الاحصائية والتنبؤية لتصنيف المقالات إلى أقسامها. سنستخدم في هذا الدرس الانحدار اللوجستي للتصنيف وذلك لسهولة تطبيقه. ولكن يجب أن تفكر باستخدام نماذج أخرى أكثر تعقيداً مثل Random Forests إن كانت دقة التصنيف عاملاً مهماً.

In [20]:
from sklearn.linear_model import LogisticRegression

In [21]:
from sklearn.model_selection import cross_val_score

<div style="direction:rtl;text-align:right">
إحدى أجمل ميزات مكتبة سايكت ليرن هو توفير أوامر تسهل الكثير من المهام. في الدرس السابق تحدثنا عن أهمية فصل عينة البيانات إلى عينة لتدريب النماذج وعينة أخرى لاختبارها فيما يعرف بـ cross_validation. ولكن المكتبة توفر أمر cross_val_score يقوم بكل هذا في الخلفية بدلاً من عملها بشكل يدوي كما رأيناها في الدرس السابق. كل مايحتاجه الأمر هو المصنف، والبيانات.

In [22]:
lg = LogisticRegression()
socres = cross_val_score(lg, text_doc_matrix, df.category, scoring='accuracy')

/Users/Home/anaconda/envs/rl/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/Home/anaconda/envs/rl/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/Home/anaconda/envs/rl/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


<div style="direction:rtl;text-align:right">
هنا نأخذ متوسط الدرجات لنرى متوسط دقة التصنيف عموماً:

In [23]:
np.mean(socres)

0.9216467732178865

<div style="direction:rtl;text-align:right">
وهذا يعني أن المصنف قادر على توقع بشكل صحيح في 92% من الحالات فقط باستخدام أنماط تكرار الكلمات.

<div style="direction:rtl;text-align:right">
ولكن الاستفادة من هذا النموذج لا تتوقف عند هذا الحد بل يمكننا استخدام هذا النموذج لتصنيف أي نص نريده. نحتاج أولاً لتحويله إلى صيغة رياضية (وهو ما نفعله باستخدام الأمر vectorizer.transform، ثم نضع النتيجة في المودل ونطلب منه توقع صنف النص.

In [24]:
new_phrase = vectorizer.transform(['أشتريت بالأمس جهاز كمبيوتر جديد ورائع جداً'])

In [25]:
new_phrase.shape

(1, 2975)

In [26]:
new_phrase.sum()

1

<div style="direction:rtl;text-align:right">
نقوم أولاً بتدريب النموذج على جميع المقالات الموجودة لدينا عبر الأمر التالي:

In [27]:
lg.fit(text_doc_matrix, df.category)

/Users/Home/anaconda/envs/rl/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/Home/anaconda/envs/rl/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

<div style="direction:rtl;text-align:right">
ثم نستخدم predict لتوقع صنف أي نص جديد.

In [28]:
lg.predict(new_phrase)

array([2])

<div style="direction:rtl;text-align:right">
أو نستخدم predict_proba لاستخراج احتماليات انتماء النص لكل الأصناف.. الاحتمالية الأعلى هي ما نراه في الأمر السابق

In [29]:
lg.predict_proba(new_phrase)

array([[0.24820922, 0.11361141, 0.28358469, 0.23312529, 0.1214694 ]])

<div style="direction:rtl;text-align:right">
وشكراً لمتابعتك